# Folium - 00 Documentation tutorial



**References**

Folium Documentation (n.d.) Quickstart https://python-visualization.github.io/folium/quickstart.html

## 1 Load map

In [13]:
import folium

vancouver = [45.5236, -122.6750]
m = folium.Map(location=vancouver)
m

In [4]:
import os 
os.mkdir("./folium/")
m.save('./folium/index.html')

### 1-2 Some built-in map styles

#### 1-2-1 `tiles="Mapbox Bright"`

In [22]:
folium.Map(location=vancouver, tiles="Mapbox",
           API_key="key")

#### 1-2-2 `tiles="Mapbox Control Room"`

In [23]:
folium.Map(location=vancouver, tiles="Mapbox",
           API_key="key")

## 2 Markers

### 2-1 `leaflet`
- `tooltip = "message"`
- `popup = "information"`

In [28]:
m = folium.Map(location=[45.372, -121.6972], zoom_start=12, tiles="Stamen Terrain")

tooltip = "Click me!"

folium.Marker(
    [45.3288, -125.6625], popup="<h1>Random Spot at sea</h1>", tooltip=tooltip
).add_to(m)
folium.Marker(
    [45.3311, -121.7113], popup="<strong>Mountain on land</strong>", tooltip=tooltip
).add_to(m)

m

 ## 3 GeoJSON/TopoJSON Overlays

 - `tiles="cartodbpositron"`

In [29]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
antarctic_ice_edge = f"{url}/antarctic_ice_edge.json"
antarctic_ice_shelf_topo = f"{url}/antarctic_ice_shelf_topo.json"

In [ ]:
requests.get(antarctic_ice_shelf_topo).text

In [ ]:
json.loads(requests.get(antarctic_ice_shelf_topo).text)

In [32]:
import json
import requests

antarctic = [-59.1759, -11.6016]
m = folium.Map(location=antarctic,
               tiles="cartodbpositron",
               zoom_start=2)

folium.GeoJson(antarctic_ice_edge, name="geojson").add_to(m)

folium.TopoJson(json.loads(requests.get(antarctic_ice_shelf_topo).text),
                "objects.antarctic_ice_shelf",
                name="topojson").add_to(m)

folium.LayerControl().add_to(m)

m

## 4 Choropleth maps

Choropleth can be easily created by binding the data between Pandas DataFrames/Series and Geo/TopoJSON geometries.

In [37]:
state_geo

"""
{"type":"Feature","id":"AK","properties":{"name":"Alaska"},
"geometry":{"type":"MultiPolygon","coordinates":[[[[-131.602021,55.117982],[-131.569159,55.28229],[-131.355558,55.183705], ...
"""

'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json'

In [38]:
state_unemployment

"""
State,Unemployment  # feature names
AL,7.1              # ID, float 
AK,6.8
AZ,8.1
"""

'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/US_Unemployment_Oct2012.csv'

In [35]:
import pandas as pd

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"
state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)

m

## 5 Styling

In [40]:
county_data

'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us_county_data.csv'

In [39]:
import branca

url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
county_data = f"{url}/us_county_data.csv"
county_geo = f"{url}/us_counties_20m_topo.json"

df = pd.read_csv(county_data, na_values=[" "])

colorscale = branca.colormap.linear.YlOrRd_09.scale(0, 50e3)
employed_series = df.set_index("FIPS_Code")["Employed_2011"]

def style_function(feature):
    employed = employed_series.get(int(feature["id"][-5:]), None)
    return {
        "fillOpacity": 0.5,
        "weight": 0,
        "fillColor": "#black" if employed is None else colorscale(employed),
    }


m = folium.Map(location=[48, -102], tiles="cartodbpositron", zoom_start=3)

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    "objects.us_counties_20m",
    style_function=style_function,
).add_to(m)

m